In [ ]:
import os
from pathlib import Path
import requests
import zipfile
import tarfile
import shutil
from warnings import filterwarnings
filterwarnings('ignore')
def download_to_cache(url, filename, force_download=False):
    def _get_custom_cache_dir():
        custom_cache_dir = Path.home() / ".cache"/ "skmini"/ "datasets"
        os.makedirs(custom_cache_dir, exist_ok=True)
        return custom_cache_dir

    file_path = _get_custom_cache_dir() / filename
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        if not force_download:
            print('File Aldready exists. Use force_download=True to force download.')
    else:
        force_download=True

    if force_download:
        try:
            if os.path.isfile(file_path) or os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Remove the directory and all its contents
                print(f"Successfully deleted the directory and its contents: {file_path}")
        except OSError as e:
            print(f"Error: {e.strerror}")

        response = requests.get(url, stream=True, verify=False)
        print(response)
        with open(file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
        print(f"Downloaded file to {file_path}")
        # now that it is downloaded, we might want to decompress the file
        if tarfile.is_tarfile(file_path):
            try:
                tar = tarfile.open(file_path)
                os.remove(file_path)
                tar.extractall(path = file_path)
                tar.close()
            except:
                print('Decompression Failed.')
        elif zipfile.is_zipfile(file_path): # change to endswith, which doesnt require us loading a library
            try:
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    print(file_path)
                    os.remove(file_path)
                    os.mkdir(file_path)
                    zip_ref.extractall(file_path)
            except: print('Decompression Failed.')


In [ ]:
from sklearn import datasets
x = datasets.load_digits()
# print(x.shape)
x
# x.reshape((-1,28,28))

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x.images[10])

In [ ]:
dir(datasets)

In [ ]:
x = [0,0,5,13,9,1,0,0,0,0,13,15,10,15,5,0,0,3,15,2,0,11,8,0,0,4,12,0,0,8,8,0,0,5,8,0,0,9,8,0,0,4,11,0,1,12,7,0,0,2,14,5,10,12,0,0,0,0,6,13,10,0,0,0,0]
len(x)

In [ ]:
from skmini.datasets import load_digits, load_diabetes, load_iris

In [ ]:
load_diabetes()

In [2]:
from skmini.datasets import load_cifar10

In [3]:
load_cifar10(force_download=True)

Successfully deleted the file
0
[----------------------------------------] 0.00%0
[----------------------------------------] 0.01%0
[----------------------------------------] 0.01%0
[----------------------------------------] 0.02%0
[----------------------------------------] 0.02%0
[----------------------------------------] 0.03%0
[----------------------------------------] 0.03%0
[----------------------------------------] 0.04%0
[----------------------------------------] 0.04%0
[----------------------------------------] 0.05%0
[----------------------------------------] 0.05%0
[----------------------------------------] 0.06%0
[----------------------------------------] 0.06%0
[----------------------------------------] 0.07%0
[----------------------------------------] 0.07%0
[----------------------------------------] 0.08%0
[----------------------------------------] 0.08%0
[----------------------------------------] 0.09%0
[----------------------------------------] 0.09%0
[-----------------

KeyboardInterrupt: 